In [1]:
import pandas as pd
from pymongo import MongoClient

from IPython.core.display import display, HTML
[display(HTML("<style>.container { width:100% !important; }</style>"))];

In [2]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["dwh"]
    
    country_code_collection = database['country_codes']
    country_code_collection.drop()
    country_code_collection.insert_many([
        {'country': data['country'], 'code': {'who': data['code_who'], 'noc': data['code_noc']}}
        for _, data in pd.read_csv('data/country_codes.csv').iterrows()
    ])

In [3]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["dwh"]
    
    who_ppp_collection = database['who_ppp_2017']
    who_ppp_collection.drop()
    data = []
    who_ppp_pc, who_ppp_abs = pd.read_csv('data/ppp_2017_pc.csv'), pd.read_csv('data/ppp_2017_abs.csv')
    for _, row in who_ppp_pc.iterrows():
        data.append({'cc': row['Country Code'], 'ppp': {'pc': float(row['Value']), 'abs': float(who_ppp_abs[who_ppp_abs['Country Code'] == row['Country Code']]['Value'])}})
    who_ppp_collection.insert_many(data)

In [4]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["dwh"]
    
    medal_wins_collection = database['medal_wins']
    medal_wins_collection.drop()
    wins_by_noc = {}
    for _, row in pd.read_csv('data/medal_wins.csv').iterrows():
        wins_by_noc[noc] = wins_by_noc.get((noc := str(row['NOC'])), []) + [row[['Name', 'Year', 'Sport', 'Medal']]]
    medal_wins_collection.insert_many([{'noc': k, 'wins': list({'name': x[0], 'year': x[1], 'sport': x[2], 'medal': x[3]} for x in v)} for k, v in wins_by_noc.items()])

In [5]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["dwh"]
    
    aggregated_data_collection = database['aggregated_data']
    aggregated_data_collection.drop()
    aggregated_data = []
    
    for country_code in country_code_collection.find():
        aggregated_data.append({
            'country': country_code['country'],
            'code': {
                'who': (cc_who := country_code['code']['who']),
                'noc': (cc_noc := country_code['code']['noc'])
            },
            'medal_wins': {
                'bronze': {
                    'by_year': (bronze := {
                        str(year): len(list(w for w in (medal_wins_collection.find_one({'noc': {'$eq': cc_noc}}) or {'wins': []})['wins'] if w['year'] == year and w['medal'] == 'Bronze')) 
                        for year in set(x['year'] for x in (medal_wins_collection.find_one({'noc': {'$eq': cc_noc}}) or {'wins': []})['wins'])
                    }),
                    'sum': sum(bronze.values())
                },
                'silver': {
                    'by_year': (silver := {
                        str(year): len(list(w for w in (medal_wins_collection.find_one({'noc': {'$eq': cc_noc}}) or {'wins': []})['wins'] if w['year'] == year and w['medal'] == 'Silver')) 
                        for year in set(x['year'] for x in (medal_wins_collection.find_one({'noc': {'$eq': cc_noc}}) or {'wins': []})['wins'])
                    }),
                    'sum': sum(silver.values())
                },
                'gold': {
                    'by_year': (gold := {
                        str(year): len(list(w for w in (medal_wins_collection.find_one({'noc': {'$eq': cc_noc}}) or {'wins': []})['wins'] if w['year'] == year and w['medal'] == 'Gold')) 
                        for year in set(x['year'] for x in (medal_wins_collection.find_one({'noc': {'$eq': cc_noc}}) or {'wins': []})['wins'])
                    }),
                    'sum': sum(gold.values())
                }
            },
            'health_expenditures': {
                'per_capita': (ppp := who_ppp_collection.find_one({'cc': {'$eq': cc_who}})['ppp'])['pc'],
                'absolute': ppp['abs'],
            }
        }) 
        
    aggregated_data_collection.insert_many(aggregated_data)